In [ ]:
from pdf2image import convert_from_path
import pytesseract
from PIL import Image, ImageFilter

# Path to the PDF file
pdf_path = '/Users/annakelmanson/Downloads/paper-1-transformed-1.pdf'

# Convert PDF to images with high DPI
images = convert_from_path(pdf_path, dpi=600)

# Iterate over each image (page)
for i, image in enumerate(images):
    # Apply preprocessing to enhance image quality
    image = image.convert('L')  # Convert to grayscale
    image = image.filter(ImageFilter.SHARPEN)  
    image = image.filter(ImageFilter.)

    # Save the preprocessed image as a JPEG file
    image_path = f'page_{i+1}.jpg'
    image.save(image_path, 'JPEG')

    # Use pytesseract to do OCR on the preprocessed image
    text = pytesseract.image_to_string(f'./enhanced_image.jpg', lang='eng')  # Specify language if needed

    # Print or save the extracted text
    print(f"Text from page {i+1}:\n{text}\n")


In [ ]:
import os
from PIL import Image, ImageFilter
import pytesseract

# Directory containing the JPG files
jpg_directory = './jpgs'

# Get all JPG files in the directory
jpg_files = [f for f in os.listdir(jpg_directory) if f.endswith('.jpg')]

# Iterate over each JPG file
for i, jpg_file in enumerate(jpg_files):
    # Open the image
    image_path = os.path.join(jpg_directory, jpg_file)
    image = Image.open(image_path)

    # Apply preprocessing to enhance image quality
    image = image.convert('L')  # Convert to grayscale
    image = image.filter(ImageFilter.SHARPEN)

    # Save the preprocessed image as a new JPEG file
    enhanced_image_path = f'enhanced_{jpg_file}'
    image.save(enhanced_image_path, 'JPEG')

    # Use pytesseract to do OCR on the preprocessed image
    text = pytesseract.image_to_string(image, lang='eng')  # Specify language if needed

    # Print or save the extracted text
    print(f"Text from {jpg_file}:\n{text}\n")

In [ ]:
from pdf2image import convert_from_path
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter

# Path to the PDF file
pdf_path = '/Users/annakelmanson/Downloads/paper-1-transformed-1.pdf'

# Convert PDF to images with high DPI
images = convert_from_path(pdf_path, dpi=600)

def enhance_images(images):
    enhanced_images = []
    for image in images:
        # Convert to grayscale
        gray_image = image.convert('L')
        
        # Increase contrast
        contrast_enhancer = ImageEnhance.Contrast(gray_image)
        contrasted_image = contrast_enhancer.enhance(1.5)  # Increase contrast by 50%
        
        # Apply sharpening filter
        sharpened_image = contrasted_image.filter(ImageFilter.SHARPEN)
        
        enhanced_images.append(sharpened_image)
    return enhanced_images

# Enhance images by increasing contrast and sharpening
enhanced_images = enhance_images(images)

# Iterate over each enhanced image (page)
for i, image in enumerate(enhanced_images):
    # Save the enhanced image as a JPEG file
    image_path = f'page_{i+1}_enhanced.jpg'
    image.save(image_path, 'JPEG')

    # Use pytesseract to do OCR on the enhanced image
    text = pytesseract.image_to_string(image, lang='eng')  # Specify language if needed

    # Print or save the extracted text
    print(f"Text from page {i+1}:\n{text}\n")


In [ ]:
import cv2
import numpy as np
from pdf2image import convert_from_path
from PIL import Image, ImageEnhance, ImageFilter

# Path to the PDF file
pdf_path = '/Users/annakelmanson/Downloads/paper-1-transformed-1.pdf'

# Convert PDF to images with high DPI
images = convert_from_path(pdf_path, dpi=500)

def enhance_image(image):
    # Convert PIL image to OpenCV format
    open_cv_image = np.array(image)
    open_cv_image = open_cv_image[:, :, ::-1].copy()  # Convert RGB to BGR

    # Convert to grayscale
    gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding (binarization)
    binary = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 11, 2
    )

    # Increase contrast using histogram equalization
    contrasted = cv2.equalizeHist(binary)

    # Convert back to PIL Image for saving or further processing
    pil_image = Image.fromarray(contrasted)

    return pil_image

# Enhance each image for better resolution and clarity
enhanced_images = [enhance_image(image) for image in images]

# Save enhanced images
for i, enhanced_image in enumerate(enhanced_images):
    image_path = f'page_{i+1}_enhanced.jpg'
    enhanced_image.save(image_path, 'JPEG')


In [ ]:
import cv2
import numpy as np
from PIL import Image

# Load the image
image_path = 'page_1.jpg'
image = cv2.imread(image_path)

# Check if the image was loaded successfully
if image is None:
    raise FileNotFoundError(f"Image not found at {image_path}")

# Step 1: Normalization
norm_img = np.zeros((image.shape[0], image.shape[1]))
normalized_img = cv2.normalize(image, norm_img, 0, 255, cv2.NORM_MINMAX)

# Step 2: Skew Correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

deskewed_img = deskew(normalized_img)

# Step 4: Noise Removal
denoised_img = cv2.fastNlMeansDenoisingColored(deskewed_img, None, 10, 10, 7, 15)

# Step 6: Grayscale Conversion
gray_img = cv2.cvtColor(denoised_img, cv2.COLOR_BGR2GRAY)

# Step 7: Thresholding/Binarization
_, binary_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Save the processed image
output_path = 'processed_image.jpg'
cv2.imwrite(output_path, binary_img)


In [ ]:


import cv2
from matplotlib import pyplot as plt
image_file = "page_1.jpg"
img = cv2.imread(image_file)

def display(im_path):
    dpi = 300
    im_data = plt.imread(im_path)

    height, width  = im_data.shape[:2]
    
    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image.
    ax.imshow(im_data, cmap='gray')

    plt.show()


inverted_image = cv2.bitwise_not(img)
cv2.imwrite("inverted.jpg", inverted_image)

def grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

gray_image = grayscale(img)
cv2.imwrite("tgray.jpg", gray_image)

thresh, im_bw = cv2.threshold(gray_image, 210, 230, cv2.THRESH_BINARY)
cv2.imwrite("bw_image.jpg", im_bw)

def noise_removal(image):
    import numpy as np
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    image = cv2.medianBlur(image, 3)
    return (image)

no_noise = noise_removal(im_bw)
cv2.imwrite("no_noise.jpg", no_noise)

def thin_font(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernel = np.ones((3,3),np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return (image)


eroded_image = thin_font(no_noise)
cv2.imwrite("eroded_image.jpg", eroded_image)


def thick_font(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernel = np.ones((1,1),np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return (image)

dilated_image = thick_font(no_noise)
cv2.imwrite("dilated_image.jpg", dilated_image)



In [67]:
import cv2
import numpy as np
image = cv2.imread('./jpgs/enhanced_image.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, th = cv2.threshold(gray,
	127,  # threshold value
	255,  # maximum value assigned to pixel values exceeding the threshold
	cv2.THRESH_BINARY) 


In [69]:
image = cv2.imread('./jpgs/enhanced_image.jpg', 0)  # read as grayscale
th = cv2.adaptiveThreshold(image, 
    255,  # maximum value assigned to pixel values exceeding the threshold
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,  # gaussian weighted sum of neighborhood
    cv2.THRESH_BINARY,  # thresholding type
    5,  # block size (5x5 window)
    3) 

In [ ]:
image = cv2.imread('book_page_1_cropped.jpg', 0)
ret, th = cv2.threshold(image,
    0,  # threshold value, ignored when using cv2.THRESH_OTSU
    255,  # maximum value assigned to pixel values exceeding the threshold
    cv2.THRESH_BINARY + cv2.THRESH_OTSU) 

In [ ]:
import pytesseract
from PIL import Image

# Load the processed image
image_path = 'piece_4_1.jpg'
image = Image.open(image_path)

# Use Tesseract to do OCR on the image
text = pytesseract.image_to_string(image)

# Print or save the extracted text
print(text)


In [ ]:
pip install pdfplumber

In [73]:
import pdfplumber
pdf = pdfplumber.open("/Users/annakelmanson/Downloads/paper-1-transformed-1.pdf")
page = pdf.pages[0]
table = page.extract_table()

In [74]:
table

In [ ]:
import cv2

# Load the image
image_path = 'page_1.jpg'
image = cv2.imread(image_path)

# Check if the image was loaded successfully
if image is None:
    raise FileNotFoundError(f"Image not found at {image_path}")

# Get the dimensions of the image
height, width = image.shape[:2]

# Define the number of rows and columns to split into
num_rows = 10  # Adjust based on your needs
num_cols = 3  # Adjust based on your needs

# Calculate height and width of each piece
piece_height = height // num_rows
piece_width = width // num_cols

# Split the image into smaller pieces and save them
for row in range(num_rows):
    for col in range(num_cols):
        y_start = row * piece_height
        y_end = (row + 1) * piece_height
        x_start = col * piece_width
        x_end = (col + 1) * piece_width
        piece = image[y_start:y_end, x_start:x_end]
        
        # Save each piece
        piece_path = f'piece_{row}_{col}.jpg'
        cv2.imwrite(piece_path, piece)

print(f"Image split into {num_rows * num_cols} pieces.")


In [ ]:
import camelot

tables = camelot.read_pdf('/Users/annakelmanson/Downloads/paper.pdf')
tables.export('output.csv', f='csv')  # Export to CSV


In [ ]:
pip install camelot